</center><img src="coverage_workflow_image.png"  width="1000" height="300"/></center>

In [3]:
# import required python libraries
#import geopandas as gpd  # for geo-data structure
import numpy as np  # for some operation better done with numpy
import matplotlib.pyplot as plt  # for creating plots
#import shapely.geometry  # for geometry data crunching
#from shapely.geometry import Point  # specifically for point geometry data manipulation
#import keplergl
#from keplergl import KeplerGl
import datacompy
import pandas as pd
crs = {'init': 'epsg:4326'}
#from shapely.geometry import Point
import mpl_toolkits


In [11]:
# bring in the 3 to 4 different coverage dataset from FB, GSMA, ITU and Ericsson as well as the master school_geolocation

In [7]:
d_f = pd.read_csv('./data/SA_fb.csv',low_memory=False,encoding='utf-8')
d_i = pd.read_csv('./data/SA_ITU.csv',low_memory=False,encoding='utf-8')
d_e = pd.read_csv('./data/SA_EC.csv',low_memory=False,encoding='utf-8')
#d_g = pd.read_csv('./data/GSMA_RW_Schools_mobile_coverage_v1.csv',low_memory=False,encoding='utf-8')
s_master = pd.read_csv('./data/SA_master.csv',low_memory=False,encoding='utf-8')

In [10]:
d_e.head(2)

,SCHOOL_CODE,SCHOOL_NAME,LATITUDE,LONGITUDE,uuid,Duplicate,Coverage,Coverage_type,2G_coverage,3G_coverage,...,3G_Best_Coverage_provider,4G_Best_Coverage_provider,5G_Best_Coverage_provider,2G_coverage_present,3G_coverage_present,4G_coverage_present,5G_coverage_present,Distance_to_nearest_tower,Data_Source,connectivity
0,G_ZAF_000024,Saint Andrews Primary School,-26.81055,30.44670,8b446cd8-cff8-362d-9862-40d6a6d500ea,0,Yes,4G,Good,Good,...,Vodacom,Telkom,None,Yes,Yes,Yes,No,-1,Ericsson,NaN
1,G_ZAF_000058,Castilian Primary School,-25.95736,30.56519,1eeb1583-f684-308a-ac0d-a0b939b2f7f3,0,Yes,4G,Good,Good,...,MTN,Cell C,None,Yes,Yes,Yes,No,-1,Ericsson,NaN


In [11]:
d_e.rename(columns={'SCHOOL_CODE':'giga_school_id', 'SCHOOL_NAME':'school_name', '2G_coverage':'2G_quality','3G_coverage':'3G_quality','4G_coverage':'4G_quality', 'Coverage':'coverage','Coverage_type': 'coverage_type'}, inplace=True)
#d_i.rename(columns={'name':'school_name'}, inplace=True)
#d_f.rename(columns={'school_id': 'giga_school_id','name':'school_name'}, inplace=True)

In [56]:
# cross-profiles the sources with master data for completeness and correctness

In [12]:
# create a separate dfs
master = s_master.copy()
fb = d_f.copy()
ec = d_e.copy()
itu = d_i.copy()
#gsma = d_g.copy()
# for facebook
cross_profile_fb_master = datacompy.Compare(
master,
fb,
join_columns='giga_school_id',  
df1_name='Master', 
df2_name='Facebook'
)
#for ericsson
cross_profile_e_master = datacompy.Compare(
master,
ec,
join_columns='giga_school_id',  
df1_name='Master', 
df2_name='Ericsson'
)
# for ITU
cross_profile_itu_master = datacompy.Compare(
master,
itu,
join_columns='giga_school_id',  
df1_name='Master', 
df2_name='ITU'
)
print(cross_profile_fb_master.report())
print(cross_profile_e_master.report())
print(cross_profile_itu_master.report())
#print(cross_profile_gsma_master.report())

DataComPy Comparison
--------------------

DataFrame Summary
-----------------

  DataFrame  Columns   Rows
0    Master       13  32925
1  Facebook       20  32925

Column Summary
--------------

Number of columns in common: 13
Number of columns in Master but not in Facebook: 0
Number of columns in Facebook but not in Master: 7

Row Summary
-----------

Matched on: giga_school_id
Any duplicates on match values: No
Absolute Tolerance: 0
Relative Tolerance: 0
Number of rows in common: 32,925
Number of rows in Master but not in Facebook: 0
Number of rows in Facebook but not in Master: 0

Number of rows with some compared columns unequal: 1,034
Number of rows with all compared columns equal: 31,891

Column Comparison
-----------------

Number of columns compared with some values unequal: 1
Number of columns compared with all values equal: 12
Total number of values which compare unequal: 1,034

Columns with Unequal Values or Types
------------------------------------

        Column Master 

In [58]:
# some colunm names standardization

In [59]:
#d_e.rename(columns={'SCHOOL_CODE':'school_id', 'SCHOOL_NAME':'school_name'}, inplace=True)
#d_i.rename(columns={'giga_schoo': 'giga_school_id','source_sch':'school_id', 'name':'school_name'}, inplace=True)
#d_g.rename(columns={'giga_schoo': 'giga_school_id'}, inplace=True)

In [60]:
# cross-profiles the sources with master data for completeness and correctness

In [61]:
# they usually don't come with "coverage" colunms, we create it at source level using the following codes

In [13]:
#d_e['coverage'] = np.where((d_e['2G_coverage']== 'None') & (d_e['3G_coverage']== 'None')& (d_e['4G_coverage']== 'None'), 'NO', 'YES')
d_f['coverage'] = np.where((d_f['percent_2g'].isnull()) & (d_f['percent_3g'].isnull())& (d_f['percent_4g'].isnull()), 'NO', 'YES')
d_f['coverage'] = np.where((d_f['percent_2g']<=0) & (d_f['percent_3g']<=0) & (d_f['percent_4g']<=0), 'NO', 'YES')
d_i['coverage'] = np.where((d_i['2G'].isnull()) & (d_i['3G'].isnull())& (d_i['4G'].isnull()), 'NO', 'YES')
d_i['coverage'] = np.where((d_i['2G']<=0) & (d_i['3G']<=0) & (d_i['4G']<=0), 'NO', 'YES')


In [63]:
# create the 2G, 3G and 4G coverage columns with binary values at source level using the following codes

In [14]:
d_f['2G_coverage'] = np.where((d_f['percent_2g'] > 0), 'YES', 'NO')
d_f['3G_coverage'] = np.where((d_f['percent_3g'] > 0), 'YES', 'NO')
d_f['4G_coverage'] = np.where((d_f['percent_4g'] > 0), 'YES', 'NO')

In [15]:
d_i['2G_coverage'] = np.where((d_i['2G'] >=1), 'YES', 'NO')
d_i['3G_coverage'] = np.where((d_i['3G'] == 1), 'YES', 'NO')
d_i['4G_coverage'] = np.where((d_i['4G'] ==1), 'YES', 'NO')

In [66]:
#d_e.rename(columns={'2G_coverage':'2G_coverage_quality', '3G_coverage':'3G_coverage_quality', '4G_coverage': '4G_coverage_quality'}, inplace=True)

In [67]:
# they usually don't come with "coverage_type" colunms, we create it at source level using the following function and codes

In [16]:
def coverage_type(df):
    if df['4G_coverage'] == 'YES':
        return "4G"
    elif df['3G_coverage'] == 'YES':
        return "3G"
    elif df['2G_coverage'] == 'YES':
        return "2G"
    else:
        return "None"

In [17]:
d_f['coverage_type'] = d_f.apply((lambda x: coverage_type(x)),axis=1)
#d_e['coverage_type'] = d_e.apply((lambda x: coverage_type(x)),axis=1)
d_i['coverage_type'] = d_i.apply((lambda x: coverage_type(x)),axis=1)


In [18]:
d_e.head(1)

,giga_school_id,school_name,LATITUDE,LONGITUDE,uuid,Duplicate,coverage,coverage_type,2G_quality,3G_quality,...,3G_Best_Coverage_provider,4G_Best_Coverage_provider,5G_Best_Coverage_provider,2G_coverage_present,3G_coverage_present,4G_coverage_present,5G_coverage_present,Distance_to_nearest_tower,Data_Source,connectivity
0,G_ZAF_000024,Saint Andrews Primary School,-26.81055,30.4467,8b446cd8-cff8-362d-9862-40d6a6d500ea,0,Yes,4G,Good,Good,...,Vodacom,Telkom,None,Yes,Yes,Yes,No,-1,Ericsson,NaN


In [19]:
d_e.rename(columns={'2G_coverage_present':'2G_coverage', '3G_coverage_present':'3G_coverage','4G_coverage_present':'4G_coverage', 'Coverage':'coverage', 'Coverage_type':'coverage_type'}, inplace=True)

In [72]:
# isolating the needed colunms from the rest

In [20]:
features = ['giga_school_id','coverage','coverage_type','2G_coverage','3G_coverage','4G_coverage'] 

In [21]:
d_fc = d_f[features]
#d_ec = d_e[features]
d_it = d_i[features]
d_ec = d_e[features]

In [31]:
# more sanity checks for nan on giga_school_id

In [22]:
d_fc = d_fc[d_fc['giga_school_id'].notna()]
#d_ec = d_ec[d_ec['giga_school_id'].notna()]
d_it = d_it[d_it['giga_school_id'].notna()]
d_ec = d_ec[d_ec['giga_school_id'].notna()]

In [28]:
d_ec.loc[d_ec['2G_coverage'] == 'Yes', '2G_coverage'] = 'YES'
d_ec.loc[d_ec['3G_coverage'] == 'Yes', '3G_coverage'] = 'YES'
d_ec.loc[d_ec['4G_coverage'] == 'Yes', '4G_coverage'] = 'YES'
d_ec.loc[d_ec['coverage'] == 'Yes', 'coverage'] = 'YES'

d_ec.loc[d_ec['2G_coverage'] == 'No', '2G_coverage'] = 'NO'
d_ec.loc[d_ec['3G_coverage'] == 'No', '3G_coverage'] = 'NO'
d_ec.loc[d_ec['4G_coverage'] == 'No', '4G_coverage'] = 'NO'
d_ec.loc[d_ec['coverage'] == 'No', 'coverage'] = 'NO'


In [29]:
d_ec.head()

,giga_school_id,coverage,coverage_type,2G_coverage,3G_coverage,4G_coverage
0,G_ZAF_000024,YES,4G,YES,YES,YES
1,G_ZAF_000058,YES,4G,YES,YES,YES
2,G_ZAF_000073,YES,4G,YES,YES,YES
3,G_ZAF_000088,YES,4G,YES,YES,YES
4,G_ZAF_000093,YES,4G,YES,YES,YES


In [76]:
# visually comparing the sources for matches and mismatches across the columns

In [30]:
df_all = pd.concat([d_fc.set_index('giga_school_id'), d_ec.set_index('giga_school_id'),d_it.set_index('giga_school_id')], 
                   axis='columns', keys=['Facebook', 'Ericsson', 'ITU'])
df_all.head()

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


Facebook                                                    \
             coverage coverage_type 2G_coverage 3G_coverage 4G_coverage   
G_ZAF_000001      YES            4G         YES         YES         YES   
G_ZAF_000002      YES            4G         YES         YES         YES   
G_ZAF_000003      YES            4G          NO         YES         YES   
G_ZAF_000004      YES            4G         YES         YES         YES   
G_ZAF_000005      YES            4G         YES         YES         YES   

             Ericsson                                                    \
             coverage coverage_type 2G_coverage 3G_coverage 4G_coverage   
G_ZAF_000001      YES            4G         YES         YES         YES   
G_ZAF_000002      YES            4G         YES         YES         YES   
G_ZAF_000003      YES            4G         YES         YES         YES   
G_ZAF_000004      YES            4G         YES         YES         YES   
G_ZAF_000005      YES            4G         YES         YES         YES   

                  ITU                                                    
             coverage coverage_type 2G_coverage 3G_coverage 4G_coverage  
G_ZAF_000001      YES            3G         YES         YES          NO  
G_ZAF_000002      YES            3G         YES         YES          NO  
G_ZAF_000003      YES            4G         YES         YES         YES  
G_ZAF_000004      YES            4G         YES         YES         YES  
G_ZAF_000005      YES            4G         YES         YES         YES

In [31]:
df_final = df_all.swaplevel(axis='columns')[d_fc.columns[1:]]
df_final.head()

coverage               coverage_type              2G_coverage  \
             Facebook Ericsson  ITU      Facebook Ericsson ITU    Facebook   
G_ZAF_000001      YES      YES  YES            4G       4G  3G         YES   
G_ZAF_000002      YES      YES  YES            4G       4G  3G         YES   
G_ZAF_000003      YES      YES  YES            4G       4G  4G          NO   
G_ZAF_000004      YES      YES  YES            4G       4G  4G         YES   
G_ZAF_000005      YES      YES  YES            4G       4G  4G         YES   

                           3G_coverage               4G_coverage                
             Ericsson  ITU    Facebook Ericsson  ITU    Facebook Ericsson  ITU  
G_ZAF_000001      YES  YES         YES      YES  YES         YES      YES   NO  
G_ZAF_000002      YES  YES         YES      YES  YES         YES      YES   NO  
G_ZAF_000003      YES  YES         YES      YES  YES         YES      YES  YES  
G_ZAF_000004      YES  YES         YES      YES  YES         YES      YES  YES  
G_ZAF_000005      YES  YES         YES      YES  YES         YES      YES  YES

In [32]:
def highlight_diff(data, color='lightcoral'):
    attr = 'background-color: {}'.format(color)
    other = data.xs('Facebook', axis='columns', level=-1)
    return pd.DataFrame(np.where(data.ne(other, level=0), attr, ''),
                        index=data.index, columns=data.columns)

#df_final.style.apply(highlight_diff, axis=None)

In [81]:
# coverage comparison report

In [33]:
# create a separate dfs
fbc = d_fc.copy()
#ecc = d_ec.copy()
ituc = d_it.copy()
ecc = d_ec.copy()

# Facebook&ITU
cross_profile_fb_itu = datacompy.Compare(
fbc,
ituc,
join_columns='giga_school_id',  
df1_name='Facebook', 
df2_name='ITU'
)

cross_profile_facebook_ec = datacompy.Compare(
fbc,
ecc,
join_columns='giga_school_id',  
df1_name='Facebook', 
df2_name='GSMA'
)


#print(cross_profile_fb_ec.report())
print(cross_profile_fb_itu.report())
#print(cross_profile_Ericsson_itu.report())
print(cross_profile_facebook_ec.report())

DataComPy Comparison
--------------------

DataFrame Summary
-----------------

  DataFrame  Columns   Rows
0  Facebook        6  32925
1       ITU        6  32925

Column Summary
--------------

Number of columns in common: 6
Number of columns in Facebook but not in ITU: 0
Number of columns in ITU but not in Facebook: 0

Row Summary
-----------

Matched on: giga_school_id
Any duplicates on match values: No
Absolute Tolerance: 0
Relative Tolerance: 0
Number of rows in common: 32,925
Number of rows in Facebook but not in ITU: 0
Number of rows in ITU but not in Facebook: 0

Number of rows with some compared columns unequal: 29,557
Number of rows with all compared columns equal: 3,368

Column Comparison
-----------------

Number of columns compared with some values unequal: 5
Number of columns compared with all values equal: 1
Total number of values which compare unequal: 62,040

Columns with Unequal Values or Types
------------------------------------

          Column Facebook dtype ITU

In [39]:
# merging and selecting the most frequent values across the sources on each row and colunm
# creating a master coverage dataset and merging with the master school location dataset

In [34]:
import statistics as stats
df_c = pd.concat([d_fc,d_ec,d_it])
coverage = df_c.groupby(['giga_school_id'],as_index=False).agg(lambda x:x.value_counts().index[0])
coverage['coverage_type'] = coverage.apply((lambda x: coverage_type(x)),axis=1)
#coverage = df_c.groupby(['giga_school_id'],as_index=False).agg(lambda x: stats.mode(x)[0][0])


In [38]:
coverage['coverage'] = np.where((coverage['2G_coverage']=='YES') | (coverage['3G_coverage']=='YES') | (coverage['4G_coverage']=='YES'), 'YES', 'NO')

In [39]:
coverage.head()

,giga_school_id,coverage,coverage_type,2G_coverage,3G_coverage,4G_coverage
0,G_ZAF_000001,YES,4G,YES,YES,YES
1,G_ZAF_000002,YES,4G,YES,YES,YES
2,G_ZAF_000003,YES,4G,YES,YES,YES
3,G_ZAF_000004,YES,4G,YES,YES,YES
4,G_ZAF_000005,YES,4G,YES,YES,YES


In [40]:
coverage[coverage['coverage']=='NO'].head(10)

,giga_school_id,coverage,coverage_type,2G_coverage,3G_coverage,4G_coverage
4049,G_ZAF_004050,NO,None,NO,NO,NO
4318,G_ZAF_004319,NO,None,NO,NO,NO
5071,G_ZAF_005072,NO,None,NO,NO,NO
5123,G_ZAF_005124,NO,None,NO,NO,NO
5156,G_ZAF_005157,NO,None,NO,NO,NO
5159,G_ZAF_005160,NO,None,NO,NO,NO
5173,G_ZAF_005174,NO,None,NO,NO,NO
5216,G_ZAF_005217,NO,None,NO,NO,NO
5920,G_ZAF_005921,NO,None,NO,NO,NO
8546,G_ZAF_008547,NO,None,NO,NO,NO


In [97]:
#coverage.loc[coverage['coverage_type'] == 'None', 'coverage_type'] = d_gs.coverage_type

In [106]:
''''
#Main
import statistics as stats
df_c = pd.concat([d_fc,d_ec,d_it])
coverage = df_c.groupby(['giga_school_id'],as_index=False).agg(lambda x: stats.mode(x)[0][0])
coverage.loc[coverage['coverage'] == 'Y', 'coverage'] = 'YES'
coverage.loc[coverage['coverage'] == 'N', 'coverage'] = 'NO'
coverage.loc[coverage['2G_coverage'] == 'Y', '2G_coverage'] = 'YES'
coverage.loc[coverage['2G_coverage'] == 'N', '2G_coverage'] = 'NO'
coverage.loc[coverage['3G_coverage'] == 'Y', '3G_coverage'] = 'YES'
coverage.loc[coverage['3G_coverage'] == 'N', '3G_coverage'] = 'NO'
coverage.loc[coverage['4G_coverage'] == 'Y', '4G_coverage'] = 'YES'
coverage.loc[coverage['4G_coverage'] == 'N', '4G_coverage'] = 'NO'

coverage.loc[coverage['coverage_type'] == '4', 'coverage_type'] = '4G'
coverage.loc[coverage['coverage_type'] == '3', 'coverage_type'] = '3G'
coverage.loc[coverage['coverage_type'] == '2', 'coverage_type'] = '2G'
coverage.loc[coverage['coverage_type'] == 'N', 'coverage_type'] = 'None'

coverage.loc[coverage['coverage_type'] == 'None', 'coverage_type'] = d_gs.coverage_type

SyntaxError: EOF while scanning triple-quoted string literal (<ipython-input-106-4b2080777d29>, line 20)

In [41]:
master.head()

,uuid,country,source,giga_school_id,school_name,latitude,longitude,hex8,dq_score,duplicate,master_timestamp,school_archive,connectivity
0,58ee6dc6-3c75-3f5c-a942-f3ea7e8b7bf7,South Africa,liquid,G_ZAF_000001,Lekwakwa Primary School,-25.62741,27.21400,88bcdd8165fffff,0.92,0,52:25.8,No,NaN
1,8cba2690-6880-3ff4-979a-5379b49591b1,South Africa,liquid,G_ZAF_000002,Sithabile Primary School,-25.07113,29.05763,88bcd12c1dfffff,1.00,0,52:25.8,No,NaN
2,f7cb5547-4273-358f-821e-7a23ea5d9c13,South Africa,liquid,G_ZAF_000003,Shishila Primary School,-25.42356,31.17813,88bcd0ccd5fffff,1.00,0,52:25.8,No,NaN
3,8d89a45d-c35b-301d-ab2c-016adc63457d,South Africa,liquid,G_ZAF_000004,Marambane Primary School,-25.09226,30.41878,88bcd02d47fffff,1.00,0,52:25.8,No,NaN
4,71fa9d0d-226d-32ee-b639-c620d074d563,South Africa,liquid,G_ZAF_000005,Masuku Primary School,-25.10655,29.05892,88bcd12eedfffff,1.00,0,52:25.8,No,NaN


In [42]:
master_w_coverage = master.merge(coverage, how="left", on=["giga_school_id"])
master_w_coverage.to_csv('./data/ZA_master_w_coverage.csv', encoding='utf-8')
master_w_coverage_connectivity = master_w_coverage.copy()
#master_w_coverage_connectivity['connectivity_availability'] = np.where((master_w_coverage_connectivity['connectivity_type'] != 'None'), 'YES', 'NO')
master_w_coverage_connectivity.loc[master_w_coverage_connectivity['coverage'] == 'NO', 'connectivity'] = 'NO'
master_w_coverage_connectivity.loc[master_w_coverage_connectivity['coverage'] == 'NO', 'coverage_type'] = 'None'
master_w_coverage_connectivity.loc[master_w_coverage_connectivity['coverage_type'] == 'None', 'coverage'] = 'NO'
master_w_coverage_connectivity.loc[master_w_coverage_connectivity['coverage_type'] == 'None', 'coverage_type'] = 'no coverage'
master_w_coverage_connectivity.to_csv('./data/ZA_master_w_coverage_connectivity.csv', encoding='utf-8')

In [43]:
master_w_coverage.head(2)

,uuid,country,source,giga_school_id,school_name,latitude,longitude,hex8,dq_score,duplicate,master_timestamp,school_archive,connectivity,coverage,coverage_type,2G_coverage,3G_coverage,4G_coverage
0,58ee6dc6-3c75-3f5c-a942-f3ea7e8b7bf7,South Africa,liquid,G_ZAF_000001,Lekwakwa Primary School,-25.62741,27.21400,88bcdd8165fffff,0.92,0,52:25.8,No,NaN,YES,4G,YES,YES,YES
1,8cba2690-6880-3ff4-979a-5379b49591b1,South Africa,liquid,G_ZAF_000002,Sithabile Primary School,-25.07113,29.05763,88bcd12c1dfffff,1.00,0,52:25.8,No,NaN,YES,4G,YES,YES,YES


In [44]:
master_w_coverage_connectivity[master_w_coverage_connectivity['coverage_type']=='no coverage'].head(10)

,uuid,country,source,giga_school_id,school_name,latitude,longitude,hex8,dq_score,duplicate,master_timestamp,school_archive,connectivity,coverage,coverage_type,2G_coverage,3G_coverage,4G_coverage
4049,21c2ebcb-39a2-3e6e-b08b-a67bd719b6eb,South Africa,liquid,G_ZAF_004050,Vlugfontein Intermedi?ƒ?×re Skool,-31.02915,25.14724,88bc53169bfffff,1.00,0,52:25.8,No,NO,NO,no coverage,NO,NO,NO
4318,302ef747-c34a-3146-a621-a8b54fe4133f,South Africa,liquid,G_ZAF_004319,Kgotatsano Primary School,-27.82904,23.92924,88adb65635fffff,1.00,0,52:25.8,No,NO,NO,no coverage,NO,NO,NO
5071,58e50920-348a-3f85-951c-44372031375d,South Africa,liquid,G_ZAF_005072,Nourivier Met Prim?ƒ?×re Skool,-30.21398,18.13675,88ad12cac1fffff,1.00,0,52:25.8,No,NO,NO,no coverage,NO,NO,NO
5123,d6717c45-f06a-34b9-82f7-684b72eab5dc,South Africa,liquid,G_ZAF_005124,Soebatsfontein Ngk Prim?ƒ?×re Skool,-30.11783,17.59525,88ad120287fffff,1.00,0,52:25.8,No,NO,NO,no coverage,NO,NO,NO
5156,4f10c5ec-55c1-3028-af08-5ff1b942fc9f,South Africa,liquid,G_ZAF_005157,Paulshoek Met Prim?ƒ?×re Skool,-30.36702,18.25582,88ad12da0bfffff,1.00,0,52:25.8,No,NO,NO,no coverage,NO,NO,NO
5159,f4f0d799-76bc-3ac9-b82c-f27e5f99383b,South Africa,liquid,G_ZAF_005160,Melton Wold Prim?ƒ?×re Skool,-31.45851,22.75667,88bc5c2729fffff,1.00,0,52:25.8,No,NO,NO,no coverage,NO,NO,NO
5173,9aec0632-c345-3080-bd5c-baaf2ba2d266,South Africa,liquid,G_ZAF_005174,Goodhouse NG Kerk Prim?ƒ?×re Skool,-28.91252,18.21078,88ada1759dfffff,1.00,0,52:25.8,No,NO,NO,no coverage,NO,NO,NO
5216,fb922ef2-5c12-3eb4-9e12-d0a235e83341,South Africa,liquid,G_ZAF_005217,Saamstaan Prim?ƒ?×re Skool,-29.64957,23.41340,88bc5a0c63fffff,0.96,0,52:25.8,No,NO,NO,no coverage,NO,NO,NO
5920,dc94e2aa-2007-38fd-9695-d0b2de212ef0,South Africa,liquid,G_ZAF_005921,Meesters House,-33.35980,21.66836,88bc41060dfffff,0.62,1,52:25.8,No,NO,NO,no coverage,NO,NO,NO
8546,b0655302-0534-38f5-b5b0-a36f90d6cb35,South Africa,liquid,G_ZAF_008547,Keerom Sskv Primary School,-33.56914,19.68895,88bc6b60a9fffff,1.00,0,52:25.8,No,NO,NO,no coverage,NO,NO,NO


In [45]:
master_w_coverage_connectivity.head()

,uuid,country,source,giga_school_id,school_name,latitude,longitude,hex8,dq_score,duplicate,master_timestamp,school_archive,connectivity,coverage,coverage_type,2G_coverage,3G_coverage,4G_coverage
0,58ee6dc6-3c75-3f5c-a942-f3ea7e8b7bf7,South Africa,liquid,G_ZAF_000001,Lekwakwa Primary School,-25.62741,27.21400,88bcdd8165fffff,0.92,0,52:25.8,No,NaN,YES,4G,YES,YES,YES
1,8cba2690-6880-3ff4-979a-5379b49591b1,South Africa,liquid,G_ZAF_000002,Sithabile Primary School,-25.07113,29.05763,88bcd12c1dfffff,1.00,0,52:25.8,No,NaN,YES,4G,YES,YES,YES
2,f7cb5547-4273-358f-821e-7a23ea5d9c13,South Africa,liquid,G_ZAF_000003,Shishila Primary School,-25.42356,31.17813,88bcd0ccd5fffff,1.00,0,52:25.8,No,NaN,YES,4G,YES,YES,YES
3,8d89a45d-c35b-301d-ab2c-016adc63457d,South Africa,liquid,G_ZAF_000004,Marambane Primary School,-25.09226,30.41878,88bcd02d47fffff,1.00,0,52:25.8,No,NaN,YES,4G,YES,YES,YES
4,71fa9d0d-226d-32ee-b639-c620d074d563,South Africa,liquid,G_ZAF_000005,Masuku Primary School,-25.10655,29.05892,88bcd12eedfffff,1.00,0,52:25.8,No,NaN,YES,4G,YES,YES,YES


In [46]:
master_w_coverage_connectivity['coverage'].value_counts()

YES    32899
NO        26
Name: coverage, dtype: int64